# Run an initial analysis of results and produce aggregated results files

In [1]:
import pandas as pd
import itertools
import matplotlib.pyplot as plt
from IPython.display import display_html

pd.options.display.max_rows = 400
from metadata_utils import get_metadata, get_tuned_alg_perf, process_metafeatures, compute_feature_corrs

In [2]:
dataset_version = ""

# For choosing metafeatures
filter_families = [
    'general',
    'statistical',
    'info-theory'
]

metadataset_df, metafeatures_df = get_metadata(dataset_version)

metafeatures_processed = process_metafeatures(metafeatures_df, filter_families=filter_families)
metafeatures_df = metafeatures_processed

/home/shared/tabzilla/tabzilla_analysis/metadata_utils.py:56: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  metafeatures_processed = metafeatures_df.fillna(metafeatures_df.median())


In [3]:
# keep only binary and classification datasets
print(metadataset_df["target_type"].value_counts())

metadataset_df = metadataset_df.loc[metadataset_df["target_type"].isin(["binary", "classification"]), :]

binary            565310
classification    417230
regression        102790
Name: target_type, dtype: int64


In [4]:
# check that all dataset-alg pairs have results for all 10 folds
print(metadataset_df.groupby(["alg_name", "dataset_name"]).agg({"dataset_fold_id": lambda x: len(set(x))}))

                                                   dataset_fold_id
alg_name    dataset_name                                          
CatBoost    openml__APSFailure__168868                          10
            openml__Amazon_employee_access__34539               10
            openml__Australian__146818                          10
            openml__Bioresponse__9910                           10
            openml__Census-Income__168340                       10
...                                                            ...
rtdl_ResNet openml__walking-activity__9945                      10
            openml__wall-robot-navigation__9960                 10
            openml__wdbc__9946                                  10
            openml__wilt__146820                                10
            openml__yeast__145793                               10

[3237 rows x 1 columns]


## Analyze num. results per dataset and alg

In [5]:
# for each alg, for how many datasets are there results?
print(f"for each alg: number of datasets with results (out of {len(metadataset_df['dataset_name'].unique())})")
print(metadataset_df.groupby("alg_name")["dataset_name"].apply(lambda x: len(set(x))).sort_values())

print(f"for each dataset: number of algs with results (out of {len(metadataset_df['alg_name'].unique())})")
metadataset_df.groupby("dataset_name")["alg_name"].apply(lambda x: len(set(x))).sort_values()

for each alg: number of datasets with results (out of 176)
alg_name
TabPFNModel            63
NAM                    80
DeepFM                 90
SAINT                 106
TabTransformer        124
NODE                  138
rtdl_FTTransformer    139
SVM                   143
DANet                 147
VIME                  163
STG                   164
CatBoost              165
LightGBM              165
KNN                   167
LinearModel           168
TabNet                168
RandomForest          173
XGBoost               174
rtdl_ResNet           174
MLP                   175
DecisionTree          175
rtdl_MLP              176
Name: dataset_name, dtype: int64
for each dataset: number of algs with results (out of 22)


dataset_name
openml__poker-hand__9890                                   5
openml__covertype__7593                                    7
openml__albert__189356                                     8
openml__Devnagari-Script__167121                           8
openml__helena__168329                                     9
openml__CIFAR_10__167124                                  10
openml__walking-activity__9945                            11
openml__airlines__189354                                  11
openml__Fashion-MNIST__146825                             11
openml__guillermo__168337                                 12
openml__riccardo__168338                                  12
openml__mnist_784__3573                                   12
openml__robert__168332                                    12
openml__Census-Income__168340                             12
openml__ldpa__9974                                        13
openml__higgs__146606                                     14
openml__ski

## Remove datasets with few results

In [6]:
print(f"for each dataset: number of algs with results")
alg_counts = metadataset_df.groupby("dataset_name")["alg_name"].agg(lambda x: len(set(x))).sort_values()
print(alg_counts)

for each dataset: number of algs with results
dataset_name
openml__poker-hand__9890                                   5
openml__covertype__7593                                    7
openml__albert__189356                                     8
openml__Devnagari-Script__167121                           8
openml__helena__168329                                     9
openml__CIFAR_10__167124                                  10
openml__walking-activity__9945                            11
openml__airlines__189354                                  11
openml__Fashion-MNIST__146825                             11
openml__guillermo__168337                                 12
openml__riccardo__168338                                  12
openml__mnist_784__3573                                   12
openml__robert__168332                                    12
openml__Census-Income__168340                             12
openml__ldpa__9974                                        13
openml__higgs__146606     

In [7]:
keep_datasets = list(alg_counts[alg_counts >= 10].index)
drop_datasets = alg_counts[alg_counts < 10]

print(f"dropping {len(drop_datasets)} datasets:")
print(drop_datasets)

dropping 5 datasets:
dataset_name
openml__poker-hand__9890            5
openml__covertype__7593             7
openml__albert__189356              8
openml__Devnagari-Script__167121    8
openml__helena__168329              9
Name: alg_name, dtype: int64


In [8]:
print("for each alg: number of datasets with results")
dataset_counts = metadataset_df.groupby("alg_name")["dataset_name"].agg(lambda x: len(set(x))).sort_values()
print(dataset_counts)

for each alg: number of datasets with results
alg_name
TabPFNModel            63
NAM                    80
DeepFM                 90
SAINT                 106
TabTransformer        124
NODE                  138
rtdl_FTTransformer    139
SVM                   143
DANet                 147
VIME                  163
STG                   164
CatBoost              165
LightGBM              165
KNN                   167
LinearModel           168
TabNet                168
RandomForest          173
XGBoost               174
rtdl_ResNet           174
MLP                   175
DecisionTree          175
rtdl_MLP              176
Name: dataset_name, dtype: int64


We will keep all algs, regardless of how many datasets they have results for.

In [9]:
metadataset_df = metadataset_df.loc[metadataset_df["dataset_name"].isin(keep_datasets), :]

print("after removing datasets: number of datasets with results")
dataset_counts = metadataset_df.groupby("alg_name")["dataset_name"].agg(lambda x: len(set(x))).sort_values()
print(dataset_counts)

after removing datasets: number of datasets with results
alg_name
TabPFNModel            63
NAM                    80
DeepFM                 90
SAINT                 106
TabTransformer        122
NODE                  138
rtdl_FTTransformer    139
SVM                   143
DANet                 147
VIME                  162
CatBoost              163
STG                   163
KNN                   164
LightGBM              164
TabNet                166
LinearModel           167
RandomForest          170
MLP                   170
rtdl_ResNet           170
XGBoost               171
DecisionTree          171
rtdl_MLP              171
Name: dataset_name, dtype: int64


# Get tuned algorithms for a given metric

Report the average & median test performance, over all folds. Note that each alg is tuned for each fold separately.

In [10]:
metric_list = [
    "Accuracy",
    "F1",
    "Log Loss",
]

obj_type_list = [
    "maximize",
    "maximize",
    "minimize",
]
result_df_dict = {}



In [11]:
# add a copy of each "default" hparam row, to treat this as a separate alg
default_rows = metadataset_df.loc[metadataset_df["hparam_source"] == "default"].copy()
default_rows.loc[:, "alg_name"] = default_rows["alg_name"].apply(lambda x: x + "_default")

# remove TabPFN and LinearModel, since these only have one hparam set
default_rows = default_rows.loc[~(default_rows["alg_name"].str.contains("TabPFNModel") | default_rows["alg_name"].str.contains("LinearModel")), :]

# append these to the metadataset
metadataset_df = pd.concat([metadataset_df, default_rows], ignore_index=True)

In [13]:
tuned_result_dfs = {}
for drop_default in [True, False]:
    for i, (metric, objective_type) in enumerate(zip(metric_list, obj_type_list)):

        test_metric_col = metric + "__test"

        if drop_default:
            df = metadataset_df.loc[~metadataset_df["alg_name"].str.contains("_default"), :].copy()
        else:
            df = metadataset_df.copy()

        tuned_alg_perf = get_tuned_alg_perf(df, metric=metric)
        # NOTE: this "tunes" each algorithm for each training fold separately. so each of the 10 folds might use different hparams.
        tuned_result_dfs[metric] = tuned_alg_perf

        ##############################
        ### STEP 1: TREAT EACH FOLD AS SEPARATE DATASET

        result_col = test_metric_col
        
        # for each dataset, find the min and max metrics over all tuned algs
        overall_bounds = tuned_alg_perf.groupby("dataset_fold_id").agg({result_col: ["min", "max"]}).reset_index()

        # rename the multiindex cols
        new_cols = []
        for c in overall_bounds.columns:
            if c[1] == "":
                new_cols.append(c[0])
            else:
                new_cols.append("_".join(c))

        overall_bounds.columns = new_cols

        tuned_alg_perf = tuned_alg_perf.merge(overall_bounds, on="dataset_fold_id", how="left")

        # add normalized metric
        tuned_alg_perf.loc[:, "normalized_" + result_col] = (tuned_alg_perf[result_col] - tuned_alg_perf[result_col + "_min"]) / (tuned_alg_perf[result_col + "_max"] - tuned_alg_perf[result_col + "_min"])

        # rank all algs for each dataset
        ascending = False if objective_type == "maximize" else True
        
        tuned_alg_perf.loc[:, f"{metric}_rank"] = tuned_alg_perf.groupby(["dataset_fold_id"])[result_col].rank(method="min", ascending=ascending).values

        # keep these cols to merge
        merge_cols = [
            "alg_name", 
            "dataset_fold_id", 
            "normalized_" + result_col,
            f"{metric}_rank",
            result_col + "_min",
            result_col + "_max"
        ]

        if i == 0:
            fold_tuned_df = tuned_alg_perf.copy()
        else:
            fold_tuned_df = fold_tuned_df.merge(tuned_alg_perf[merge_cols], on=["alg_name", "dataset_fold_id"])

        # fold_result_df_dict[metric] = tuned_alg_perf.copy()

        ##############################
        ### STEP 2: AVERAGE OVER FOLDS

        if i == 0:
            agg_dict = {
                test_metric_col: ["median", "mean"],
                "time__train": ["median", "mean"],
                # "dataset_name": ["count"],
            }
        else:
            agg_dict = {
                test_metric_col: ["median", "mean"],
            }

        # aggregate over folds: take the mean & median performance over each fold
        agg_tuned_alg_perf = tuned_alg_perf.groupby(["alg_name", "dataset_name"]).agg(agg_dict).reset_index()

        # rename the multiindex cols
        new_cols = []
        for c in agg_tuned_alg_perf.columns:
            if c[1] == "":
                new_cols.append(c[0])
            else:
                new_cols.append("_".join(c))

        agg_tuned_alg_perf.columns = new_cols


        # define the target metric column, we will use this value for all plots
        result_col = test_metric_col + "_mean"

        # for each dataset, find the min and max metrics over all tuned algs
        overall_bounds = agg_tuned_alg_perf.groupby("dataset_name").agg({result_col: ["min", "max"]}).reset_index()

        # adjust the lower bound to be the metric for a tuned decision tree (this is the "baseline")
        # baseline_metric = agg_tuned_alg_perf.loc[agg_tuned_alg_perf["alg_name"] == "DecisionTree"].groupby("dataset_name").agg({result_col: "max"}).reset_index()

        # baseline_metric.columns = ["dataset_name", "baseline_metric"]
        
        # rename the multiindex cols
        new_cols = []
        for c in overall_bounds.columns:
            if c[1] == "":
                new_cols.append(c[0])
            else:
                new_cols.append("_".join(c))

        overall_bounds.columns = new_cols

        
        agg_tuned_alg_perf = agg_tuned_alg_perf.merge(overall_bounds, on="dataset_name", how="left") #. \
            # merge(baseline_metric, on="dataset_name", how="left")

        # add normalized metric
        agg_tuned_alg_perf.loc[:, "normalized_" + result_col] = (agg_tuned_alg_perf[result_col] - agg_tuned_alg_perf[result_col + "_min"]) / (agg_tuned_alg_perf[result_col + "_max"] - agg_tuned_alg_perf[result_col + "_min"])

        # rank all algs for each dataset
        ascending = False if objective_type == "maximize" else True
        
        # rank according to mean or median performance over all folds
        for agg_method in ["mean", "median"]:
            agg_tuned_alg_perf.loc[:, f"{metric}_rank" + "_" + agg_method] = agg_tuned_alg_perf.groupby(["dataset_name"])[test_metric_col + "_" + agg_method].rank(method="min", ascending=ascending).values

        # keep these cols to merge
        merge_cols = [
            "alg_name", 
            "dataset_name",
            "normalized_" + result_col,
            f"{metric}_rank_median",
            f"{metric}_rank_mean",
            result_col + "_min",
            result_col + "_max"
        ]

        if i == 0:
            tuned_agg_df = agg_tuned_alg_perf.copy()
        else:
            tuned_agg_df = tuned_agg_df.merge(agg_tuned_alg_perf[merge_cols], on=["alg_name", "dataset_name"])

        # result_df_dict[metric] = agg_tuned_alg_perf.copy()

    # save all results

    if drop_default:
        tuned_agg_df_no_default = tuned_agg_df.copy()
        tuned_agg_df_no_default.to_csv("./results/tuned_aggregated_results.csv")

        tuned_fold_df_no_default = fold_tuned_df.copy()
        tuned_fold_df_no_default.to_csv("./results/tuned_fold_results.csv")
       
    else:
        tuned_agg_df_with_default = tuned_agg_df.copy()
        tuned_agg_df_with_default.to_csv("./results/tuned_aggregated_results_with_default_hparams.csv")

        tuned_fold_df_with_default = fold_tuned_df.copy()
        tuned_fold_df_with_default.to_csv("./results/tuned_fold_results_with_default_hparams.csv")


In [81]:
# sanity check..
# result_df_dict["Accuracy"][result_df_dict["Accuracy"]["dataset_name"] == "openml__Amazon_employee_access__34539"]
# tuned_agg_df_with_default[(tuned_agg_df_with_default["dataset_name"] == "openml__Amazon_employee_access__34539") & tuned_agg_df_with_default["alg_name"].str.contains("CatBoost")]

# tuned_agg_df_with_default[(tuned_agg_df_with_default["dataset_name"] == "openml__ada_agnostic__3896") & tuned_agg_df_with_default["alg_name"].str.contains("CatBoost")]

tuned_fold_df_no_default[fold_tuned_df["dataset_fold_id"] == "openml__APSFailure__168868__fold_1"]
# fold_tuned_df[fold_tuned_df["dataset_fold_id"] == "openml__Amazon_employee_access__34539__fold_1"]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


,results_bucket_path_x,dataset_fold_id,dataset_name_x,target_type_x,alg_name,hparam_source_x,trial_number_x,alg_hparam_id_x,exp_name_x,time__train_x,...,MSE__train,R2__train,MSE__val,R2__val,MSE__test,R2__test,Log Loss__test_min,Log Loss__test_max,normalized_Log Loss__test,Log Loss_rank
1,results/openml__APSFailure__168868/CatBoost/gp...,openml__APSFailure__168868__fold_1,openml__APSFailure__168868,binary,CatBoost,random_3_s0,3,CatBoost__seed_0__trial_3,gpu-expt-a_091822_065111_fdd9.zip,3.450149,...,NaN,NaN,NaN,NaN,NaN,NaN,0.021874,0.622607,0.003518,3.0
1631,results/openml__Amazon_employee_access__34539/...,openml__Amazon_employee_access__34539__fold_1,openml__Amazon_employee_access__34539,binary,DANet,default,0,DANet__seed_0__trial_0,algs-gpu-2-datasets-a_120522_203421_75ad.zip,397.622284,...,NaN,NaN,NaN,NaN,NaN,NaN,0.154622,0.347718,0.322875,11.0
6191,results/openml__colic__25/KNN/algs-cpu-1-datas...,openml__colic__25__fold_1,openml__colic__25,binary,KNN,random_26_s0,26,KNN__seed_0__trial_26,algs-cpu-1-datasets-b_111022_191525_9de4.zip,0.000928,...,NaN,NaN,NaN,NaN,NaN,NaN,0.149106,0.610076,0.741160,18.0
7901,results/openml__diabetes__37/LightGBM/cpu-expt...,openml__diabetes__37__fold_1,openml__diabetes__37,binary,LightGBM,random_12_s0,12,LightGBM__seed_0__trial_12,cpu-expt_092822_003611_481f.zip,0.062370,...,NaN,NaN,NaN,NaN,NaN,NaN,0.553124,0.736261,0.000000,1.0
9611,results/openml__eeg-eye-state__14951/LinearMod...,openml__eeg-eye-state__14951__fold_1,openml__eeg-eye-state__14951,binary,LinearModel,default,0,LinearModel__seed_0__trial_0,cpu-expt_091722_173804_d9a4.zip,0.017860,...,NaN,NaN,NaN,NaN,NaN,NaN,0.124636,19.044746,0.027229,7.0
10511,results/openml__synthetic_control__3512/Linear...,openml__synthetic_control__3512__fold_1,openml__synthetic_control__3512,classification,LinearModel,default,0,LinearModel__seed_0__trial_0,algs-cpu-1-datasets-b_111022_185705_4390.zip,0.003271,...,NaN,NaN,NaN,NaN,NaN,NaN,0.026999,1.018716,0.114086,11.0
11411,results/openml__gas-drift__9986/MLP/algs-gpu-1...,openml__gas-drift__9986__fold_1,openml__gas-drift__9986,classification,MLP,random_25_s0,25,MLP__seed_0__trial_25,algs-gpu-1-datasets-b-v3_111922_040534_84da.zip,34.159005,...,NaN,NaN,NaN,NaN,NaN,NaN,0.028334,20.823773,0.021742,12.0
13051,results/openml__socmob__3797/NAM/algs-gpu-2-da...,openml__socmob__3797__fold_1,openml__socmob__3797,binary,NAM,random_17_s0,17,NAM__seed_0__trial_17,algs-gpu-2-datasets-b_112822_043929_2251.zip,33.489616,...,NaN,NaN,NaN,NaN,NaN,NaN,0.118054,1.050778,0.307651,19.0
14691,results/openml__PhishingWebsites__14952/Random...,openml__PhishingWebsites__14952__fold_1,openml__PhishingWebsites__14952,binary,RandomForest,random_24_s0,24,RandomForest__seed_0__trial_24,cpu-expt_092722_213005_e9ad.zip,0.551150,...,NaN,NaN,NaN,NaN,NaN,NaN,0.065170,0.200353,0.431949,14.0
16331,results/openml__adult-census__3953/SAINT/algs-...,openml__adult-census__3953__fold_1,openml__adult-census__3953,binary,SAINT,random_2_s0,2,SAINT__seed_0__trial_2,algs-gpu-2-datasets-a_120222_161006_4b9a.zip,130.090155,...,NaN,NaN,NaN,NaN,NaN,NaN,0.285522,26.222314,0.002357,12.0


## Difference between best neural and best non-neural method

In [40]:
# now tune by algorithm type. first define the type as "neural" or "non-neural"
neural_algs = [
    "MLP",
    "TabNet",
    "VIME",
    "TabTransformer",
    "NODE",
    "STG",
    "NAM",
    "DeepFM",
    "SAINT",
    "DANet",
    "rtdl_MLP",
    "rtdl_ResNet",
    "rtdl_FTTransformer",
]

metadataset_df.loc[:, "alg_type"] = "non-neural"
metadataset_df.loc[metadataset_df["alg_name"].isin(neural_algs), "alg_type"] = "neural"

tuned_df = get_tuned_alg_perf(metadataset_df, metric=metric, group_col="alg_type")



In [17]:
# for each dataset fold, get difference between tuned neural and non-neural method (neural - non-neural)
neural_non_neural_comparison = pd.pivot(tuned_df, index="dataset_fold_id", columns=["alg_type"], values=["Accuracy__test", "F1__test", "MSE__test", "Log Loss__test", "alg_name", "time__train", "time__test"])
print(neural_non_neural_comparison.head())

                                   Accuracy__test             F1__test  \
alg_type                                   neural non-neural    neural   
dataset_fold_id                                                          
openml__APSFailure__168868__fold_0       0.992763   0.995263  0.992763   
openml__APSFailure__168868__fold_1       0.988684   0.992237  0.988684   
openml__APSFailure__168868__fold_2       0.990395   0.993947  0.990395   
openml__APSFailure__168868__fold_3       0.992368   0.995526  0.992368   
openml__APSFailure__168868__fold_4       0.991184   0.995789  0.991184   

                                              MSE__test             \
alg_type                           non-neural    neural non-neural   
dataset_fold_id                                                      
openml__APSFailure__168868__fold_0   0.995263       NaN        NaN   
openml__APSFailure__168868__fold_1   0.992237       NaN        NaN   
openml__APSFailure__168868__fold_2   0.993947       NaN  

In [18]:
# save the differences between neural and non-neural algs

# first rename the multiindex cols
new_cols = []
for c in neural_non_neural_comparison.columns:
    if c[1] == "":
        new_cols.append(c[0])
    else:
        new_cols.append("_".join(c))

neural_non_neural_comparison.columns = new_cols 
neural_non_neural_comparison.to_csv("./results/neural_non_neural_comparison.csv")

# Aggregate results

In [78]:
###### with default hparams treated as algs

# best, worst, and average performance for each alg, over all datasets
for metric in metric_list:

    overall_ranks = tuned_agg_df_with_default.groupby("alg_name").agg(
        {
            f"{metric}_rank_mean": ["min", "max", "mean", "count"],
            f"normalized_{metric}__test_mean": "mean",
        }
    ).reset_index().sort_values([(f"{metric}_rank_mean", "mean")])

    # format min/max rank columns to be ints

    overall_ranks.loc[:, "count"] = overall_ranks.loc[:, (f"{metric}_rank_mean", "count")].astype(int)
    overall_ranks.drop(columns=(f"{metric}_rank_mean", "count"), inplace=True)

    # overall_ranks.loc[:, "alg_name"] = overall_ranks.loc[:, "alg_name"].apply(lambda x: "\rot{" + x + "}")
    overall_ranks.loc[:, (f"{metric}_rank_mean", "min")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "min")].astype(int)
    overall_ranks.loc[:, (f"{metric}_rank_mean", "max")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "max")].astype(int)

    overall_ranks.loc[:, (f"{metric}_rank_mean", "mean")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "mean")].round(2)
    
    overall_ranks.loc[:, (f"normalized_{metric}__test_mean", "mean")] = overall_ranks.loc[:,(f"normalized_{metric}__test_mean", "mean")].round(2)

    print(f"metric: {metric}")
    final_table = overall_ranks.set_index("alg_name")
    print(final_table)

    # save to csv
    final_table.to_csv(f"./results/rank_tables_with_untuned_{metric}.csv", index=True)

    # save to latex
    final_table.to_latex(f"./results/rank_tables_with_untuned_{metric}.tex", index=True, escape=False)


    print("\n")

metric: Accuracy
                           Accuracy_rank_mean             \
                                          min max   mean   
alg_name                                                   
TabPFNModel                                 1  36   8.14   
CatBoost                                    1  36   8.37   
XGBoost                                     1  37   9.47   
CatBoost_default                            1  33  11.44   
rtdl_ResNet                                 1  38  11.45   
XGBoost_default                             1  40  11.49   
LightGBM                                    1  39  11.84   
SAINT                                       1  36  12.35   
LightGBM_default                            1  39  12.78   
NODE                                        1  38  12.80   
RandomForest                                1  38  13.66   
rtdl_FTTransformer                          1  32  13.70   
rtdl_ResNet_default                         1  41  14.16   
SVM                    

In [79]:
###### STEP 2: aggregate over all folds

# best, worst, and average performance for each alg, over all datasets
for metric in metric_list:

    overall_ranks = tuned_agg_df_no_default.groupby("alg_name").agg(
        {
            f"{metric}_rank_mean": ["min", "max", "mean", "count"],
            f"normalized_{metric}__test_mean": "mean",
        }
    ).reset_index().sort_values([(f"{metric}_rank_mean", "mean")])

    # format min/max rank columns to be ints

    overall_ranks.loc[:, "count"] = overall_ranks.loc[:, (f"{metric}_rank_mean", "count")].astype(int)
    overall_ranks.drop(columns=(f"{metric}_rank_mean", "count"), inplace=True)

    # overall_ranks.loc[:, "alg_name"] = overall_ranks.loc[:, "alg_name"].apply(lambda x: "\rot{" + x + "}")
    overall_ranks.loc[:, (f"{metric}_rank_mean", "min")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "min")].astype(int)
    overall_ranks.loc[:, (f"{metric}_rank_mean", "max")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "max")].astype(int)

    overall_ranks.loc[:, (f"{metric}_rank_mean", "mean")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "mean")].round(2)
    
    overall_ranks.loc[:, (f"normalized_{metric}__test_mean", "mean")] = overall_ranks.loc[:,(f"normalized_{metric}__test_mean", "mean")].round(2)

    print(f"metric: {metric}")
    final_table = overall_ranks.set_index("alg_name")
    print(final_table)

    # save to csv
    final_table.to_csv(f"./results/rank_tables_{metric}.csv", index=True)

    # save to latex
    final_table.to_latex(f"./results/rank_tables_{metric}.tex", index=True, escape=False)


    print("\n")

metric: Accuracy
                   Accuracy_rank_mean             \
                                  min max   mean   
alg_name                                           
TabPFNModel                         1  20   4.92   
CatBoost                            1  19   5.40   
XGBoost                             1  20   5.85   
rtdl_ResNet                         1  21   7.05   
LightGBM                            1  21   7.21   
SAINT                               1  20   7.43   
NODE                                1  21   7.82   
RandomForest                        1  19   8.29   
rtdl_FTTransformer                  1  18   8.34   
SVM                                 1  20   8.62   
DANet                               1  21   9.02   
rtdl_MLP                            1  19   9.78   
DeepFM                              1  22  11.00   
TabNet                              1  22  11.40   
MLP                                 1  20  11.66   
TabTransformer                      1  22  11.6

## UNDER CONSTRUCTION: spaghetti plot - relative performance over different datasets.

In [18]:
# which datasets to use?

result_df_dict["F1"][(result_df_dict["F1"]["alg_name"] == "CatBoost") & (result_df_dict["F1"]["metric_rank_mean"] < 2)]

KeyError: 'metric_rank_mean'

In [23]:
# openml__diabetes__37 <-- lm does well
# openml__isolet__3481
# openml__haberman__42
# openml__robert__168332

# openml__soybean__41 <-- rf does well
# openml__vowel__3022
# openml__guillermo__168337

# openml__cmc__23 <-- mlp does well
# openml__CIFAR_10__167124
# openml__Fashion-MNIST__146825
# openml__Internet-Advertisements__167125	
# openml__dilbert__168909

# openml__Australian__146818 <-- catboost
# openml__APSFailure__168868
# openml__wdbc__9946
# openml__pc1__3918
# openml__eucalyptus__2079

In [24]:
plot_datasets = [
    "openml__diabetes__37",  # <-- lm does well
    "openml__isolet__3481",
    "openml__haberman__42",
    # "openml__robert__168332", # not enough successful algs
    "openml__soybean__41", #  <-- rf does well
    "openml__vowel__3022",
    # "openml__guillermo__168337", # not enough successful algs
    "openml__cmc__23", # <-- mlp does well
    # "openml__CIFAR_10__167124",  # not enough successful algs
    # "openml__Fashion-MNIST__146825",  # not enough successful algs
    "openml__Internet-Advertisements__167125",	
    "openml__dilbert__168909",
    "openml__Australian__146818",  #<-- catboost
    "openml__APSFailure__168868",
    "openml__wdbc__9946",
    "openml__pc1__3918",
    "openml__eucalyptus__2079",
]

# names to show on the plot
plot_dataset_names = [name[len("openml__"):].split("_")[0] for name in plot_datasets]

In [25]:
# number of results for each dataset
num_alg_per_dataset = result_df_dict["F1"].groupby("dataset_name")["alg_name"].count()
num_alg_per_dataset[num_alg_per_dataset < 10].sort_values()

dataset_name
openml__Devnagari-Script__167121                          5
openml__covertype__7593                                   5
openml__helena__168329                                    5
openml__CIFAR_10__167124                                  6
openml__albert__189356                                    6
openml__guillermo__168337                                 6
openml__Fashion-MNIST__146825                             7
openml__riccardo__168338                                  7
openml__robert__168332                                    7
openml__airlines__189354                                  8
openml__mnist_784__3573                                   8
openml__higgs__146606                                     9
openml__jungle_chess_2pcs_raw_endgame_complete__167119    9
openml__numerai28.6__167120                               9
openml__skin-segmentation__9965                           9
openml__sylvine__168912                                   9
Name: alg_name, dtype: int6

In [26]:
plot_algs = agg_tuned_alg_perf["alg_name"].unique()

# gather data for the spaghetti plot
data = dict()
for i_metric, metric_name in enumerate(metric_list):
    data[metric_name] = dict()
    for alg in plot_algs:
            data[metric_name][alg] = []
            for dataset in plot_datasets:
                vals = result_df_dict[metric_name].loc[(result_df_dict[metric_name]["alg_name"] == alg) & (result_df_dict[metric_name]["dataset_name"] == dataset), f"normalized_{metric_name}__test_mean"].values
                if len(vals) != 1:
                    print(f"there's an issue with {alg}-{dataset}-{metric_name}")
                    print(vals)
                    val = None
                else:
                    val = vals[0]
                data[metric_name][alg].append(val)
        

there's an issue with CatBoost-openml__isolet__3481-Accuracy
[]
there's an issue with LightGBM-openml__dilbert__168909-Accuracy
[]
there's an issue with RandomForest-openml__haberman__42-Accuracy
[]
there's an issue with SVM-openml__soybean__41-Accuracy
[]
there's an issue with CatBoost-openml__isolet__3481-F1
[]
there's an issue with LightGBM-openml__dilbert__168909-F1
[]
there's an issue with RandomForest-openml__haberman__42-F1
[]
there's an issue with SVM-openml__soybean__41-F1
[]
there's an issue with CatBoost-openml__isolet__3481-Log Loss
[]
there's an issue with LightGBM-openml__dilbert__168909-Log Loss
[]
there's an issue with RandomForest-openml__haberman__42-Log Loss
[]
there's an issue with SVM-openml__soybean__41-Log Loss
[]


In [1]:
### plotting kwargs

plot_alg_map = {
    "XGBoost": {
        "name": "XGBoost",
        "plt-kwargs": {"marker":"x", "color":"r", "linestyle":"--"}
    },
    "CatBoost": {
        "name": "CatBoost",
        "plt-kwargs": {"marker":"+", "color":"r", "linestyle":"--"}
    },
    "LightGBM": {
        "name": "LightGBM",
        "plt-kwargs": {"marker":"d", "color":"r", "linestyle":"--"}
    },
    "SVM": {
        "name": "SVM",
        "plt-kwargs": {"marker":"v", "color":"black", "linestyle":"-"}
    },
    "KNN": {
        "name": "KNN",
        "plt-kwargs": {"marker":"^", "color":"black", "linestyle":"-"}
    },
    "DecisionTree": {
        "name": "DecisionTree",
        "plt-kwargs": {"marker":">", "color":"black", "linestyle":"-"}
    },
    "RandomForest": {
        "name": "RandomForest",
        "plt-kwargs": {"marker":"P", "color":"black", "linestyle":"-"}
    },
    "LinearModel": {
        "name": "LinearModel",
        "plt-kwargs": {"marker":"<", "color":"black", "linestyle":"-"}
    },
    "TabNet": {
        "name": "TabNet",
        "plt-kwargs": {"marker":"X", "color":"b", "linestyle":":"}
    },
    "MLP": {
        "name": "MLP",
        "plt-kwargs": {"marker":"o", "color":"b", "linestyle":":"}
    },
    "VIME": {
        "name": "VIME",
        "plt-kwargs": {"marker":"P", "color":"b", "linestyle":":"}
    },
}

plot_algs = plot_alg_map.keys()

In [2]:
import numpy as np
fig, ax = plt.subplots(len(metric_list), 1, sharex=True, figsize=(8, 5))

for i, metric in enumerate(metric_list):
    for alg in plot_algs:    
        ax[i].plot(data[metric][alg], label=alg, markersize=7, **plot_alg_map[alg]["plt-kwargs"])
    ax[i].set_ylabel(metric)

    ax[i].set_xticks(np.arange(len(plot_dataset_names)))
    ax[i].set_xticklabels(plot_dataset_names, rotation=-35, ha='left', rotation_mode='anchor')

plt.tight_layout()
plt.subplots_adjust(hspace=0.08)

plt.legend(loc="upper center", bbox_to_anchor=(0.5, 3.6), ncol=6, fontsize="small")
plt.savefig("./results/performance_spaghetti.pdf", bbox_inches='tight')
plt.show()


NameError: name 'plt' is not defined